In [4]:
import numpy as np
import pickle
import pandas as pd
import tensorflow as tf
from tensorflow.keras import models,layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
import cv2
import os

def load_images_from_folder(folder, label, max_images=3500):
    images = []
    labels = []
    all_files = os.listdir(folder)

    # Shuffle and select a subset of files
    np.random.shuffle(all_files)
    max_images = max_images or len(all_files) // 2

    for filename in all_files[:max_images]:
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
        if img is not None:
            resized_img = cv2.resize(img, (128, 128))  # Resize to 128x128
            images.append(resized_img)
            labels.append(label)

    # Convert to NumPy arrays and add channel dimension
    images = np.array(images).reshape(-1, 128, 128, 1)
    labels = np.array(labels)

    return images, labels

# Load the images
real_images, real_labels = load_images_from_folder(r'D:\SIH project\dataset2\Dataset\Train\Real', 0)
fake_images, fake_labels = load_images_from_folder(r'D:\SIH project\dataset2\Dataset\Train\Fake', 1)

# Combine real and fake data
x = np.concatenate((real_images, fake_images), axis=0)
y = np.concatenate((real_labels, fake_labels), axis=0)

xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2,random_state=42)

# Normalize the pixel values
xtrain = xtrain / 255.0
xtest = xtest/255.0

print(f"Training data shape: {xtrain.shape}, Labels shape: {ytrain.shape}")

Training data shape: (5600, 128, 128, 1), Labels shape: (5600,)


In [8]:
##using CNN model - Test One

from tensorflow.keras import models, layers

# Define the model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),  # Grayscale input
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [9]:

model.fit(xtrain,ytrain,validation_data=(xtest,ytest),epochs=10)

Epoch 1/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 46s 250ms/step - accuracy: 0.5571 - loss: 0.6841 - val_accuracy: 0.6986 - val_loss: 0.5905
Epoch 2/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 44s 253ms/step - accuracy: 0.7241 - loss: 0.5443 - val_accuracy: 0.7621 - val_loss: 0.4882
Epoch 3/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 49s 279ms/step - accuracy: 0.7888 - loss: 0.4555 - val_accuracy: 0.7521 - val_loss: 0.4987
Epoch 4/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 52s 299ms/step - accuracy: 0.8072 - loss: 0.4090 - val_accuracy: 0.8150 - val_loss: 0.3997
Epoch 5/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 46s 261ms/step - accuracy: 0.8455 - loss: 0.3392 - val_accuracy: 0.8314 - val_loss: 0.3718
Epoch 6/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 45s 255ms/step - accuracy: 0.8661 - loss: 0.2957 - val_accuracy: 0.8521 - val_loss: 0.3371
Epoch 7/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 49s 278ms/step - accuracy: 0.8973 - loss: 0.2443 - val_accuracy: 0.8607 - val_loss: 0.3198
Epoch 8/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 49s 282ms/step - accuracy: 0.9140 - loss: 0

In [10]:
model.evaluate(xtest,ytest)
# model.save('datathonModel.h5')

44/44 ━━━━━━━━━━━━━━━━━━━━ 4s 88ms/step - accuracy: 0.8756 - loss: 0.3225


[0.3203279376029968, 0.8742856979370117]

In [ ]:

# loadedModel = load_model('datathonModel.h5')

In [12]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2

def make_gradcam_heatmap(model, img_array, layer_name='conv2d_3'):
    """
    Generate a Grad-CAM heatmap for the given image and model.
    Arguments:
    - model: Trained CNN model.
    - img_array: Preprocessed input image array.
    - layer_name: Name of the last convolutional layer for Grad-CAM.
    Returns:
    - heatmap: A 2D heatmap showing important regions for prediction.
    """
    # Create a model that maps input to the desired conv layer's output and predictions
    grad_model = tf.keras.models.Model(
        [model.input], 
        [model.get_layer(layer_name).output, model.output]
    )

    # Get the output feature maps and the model's prediction for the input image
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, 0]  # Only for the first output (binary classification)

    # Compute gradients of the predicted class with respect to the conv layer output
    grads = tape.gradient(loss, conv_outputs)

    # Pool the gradients to get the importance of each channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # Multiply each channel by the pooled gradients to get weighted activation
    conv_outputs = conv_outputs[0]  # Remove batch dimension
    heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_outputs), axis=-1)

    # Normalize the heatmap
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    
    # heatmap is already a NumPy array, so just return it
    return heatmap # Changed from heatmap.numpy() to heatmap

def display_gradcam(img, heatmap, alpha=0.4):
    """
    Superimpose the Grad-CAM heatmap on the original image.
    Arguments:
    - img: Original image.
    - heatmap: 2D Grad-CAM heatmap.
    - alpha: Opacity factor for heatmap overlay.
    """
    # Resize the heatmap to match the input image size
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))

    # Convert heatmap to RGB
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    # Convert the grayscale image to RGB to match the heatmap's channels
    # Convert img to uint8 before applying cvtColor
    img = np.uint8(img * 255)  # Convert to 8-bit unsigned integer
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)  # Convert grayscale to RGB

    # Superimpose heatmap on the original image
    superimposed_img = heatmap * alpha + img

    plt.imshow(superimposed_img.astype('uint8'))
    plt.axis('off')
    plt.show()
# Example usage:
def generate_gradcam_for_test_sample(sample_index=0):
    """
    Generate Grad-CAM visualization for a test sample.
    Arguments:
    - sample_index: Index of the test image for Grad-CAM visualization.
    """
    # Select a test image and its corresponding label
    test_image = xtest[sample_index]
    test_image_expanded = np.expand_dims(test_image, axis=0)  # Add batch dimension

    # Get Grad-CAM heatmap
    heatmap = make_gradcam_heatmap(model, test_image_expanded, layer_name='conv2d_3')

    # Display the original image and Grad-CAM overlay
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(test_image.squeeze(), cmap='gray')
    plt.title("Original Image")
    plt.axis('off')

    plt.subplot(1, 2, 2)
    display_gradcam(test_image.squeeze(), heatmap)
    plt.title("Grad-CAM Overlay")

# Generate Grad-CAM visualization for the first test imagek
generate_gradcam_for_test_sample(1)
print(model.predict(xtest[0].reshape(1,128,128,1)))

ValueError: The layer sequential_1 has never been called and thus has no defined input.

In [ ]:
print(model.predict(xtest[0].reshape(1,128,128,1)))
print(ytest[0])

ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "conv2d" is incompatible with the layer: expected axis -1 of input shape to have value 3, but received input with shape (1, 128, 128, 1)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(1, 128, 128, 1), dtype=float32)
  • training=False
  • mask=None

In [ ]:
confidence_score  = model.predict(xtest[0].reshape(1,128,128,1))
predicted_class = int(confidence_score>=0.5)
# print(f"Confidence = {confidence_score:.4f}")
print(predicted_class)
print(ytest[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1
1


C:\Users\Ajay\AppData\Local\Temp\ipykernel_2840\3857035851.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predicted_class = int(confidence_score>=0.5)


In [40]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
import cv2

# Load the image and convert it to grayscale
img_path = r'D:\SIH project\dataset2\Dataset\Train\Fake\fake_26.jpg'
def imgToArray(img_path):
    img = image.load_img(img_path, target_size=(128, 128))  # Resize
    img_array = image.img_to_array(img)

    # Convert the image to grayscale
    img_array = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)

    # Add batch dimension and normalize
    img_array = np.expand_dims(img_array, axis=(0, -1))  # Shape: (1, 128, 128, 1)
    img_array = img_array / 255.0  # Normalize the pixel values
    return img_array
img_array = imgToArray(img_path)
# Pass the image through the model
predictions = model.predict(img_array)
print("Prediction:", predictions)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Prediction: [[0.99872625]]


In [43]:
img_path = r"D:\SIH project\modified_image.jpg"
newArray = imgToArray(img_path)

newPredictions = model.predict(newArray)
print("Prediction:", predictions)

print(predictions-newPredictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Prediction: [[0.99872625]]
[[0.00010955]]
